In [1]:
import merkletools

In [2]:
mt = merkletools.MerkleTools(hash_type="keccak_256")

mt.add_leaf('0000000000000000000000000000000000000000000000000000000000000017', True)
mt.add_leaf(["bitcoin", "blockchain"], True)

mt.make_tree()

In [47]:
leaf_value =  mt.get_leaf(0)

In [19]:
mt.get_proof(1)

[{'left': 'c737aa53bf5c1c135465bab4dc4051f2a6123e709fa2d8b349bd85a5617491ca'},
 {'right': '7ee156df5091fbef71b96557542210a9c9ca851cc85aaf60026519b4aaccf491'}]

In [20]:
mt.get_leaf(1)

'7dee6e1aa550de37364ec77e03e62ea56bf42037b8297280de9d844d88444e4d'

In [21]:
mt.get_merkle_root()

'b2f0283f3d9e254bd18e12ef6da55e5c2373c2577e5e07f3157aa608d58443d8'

In [19]:
import sha3
from eth_abi.abi import encode_abi
from web3 import Web3

sha3.keccak_256(encode_abi(['int32'], [23])).hexdigest()

bytes(Web3.solidityKeccak(['int256'], [23])).hex()

##sha3.keccak_256(bytearray.fromhex('0000000000000000000000000000000000000000000000000000000000000017')).digest()

'c624b66cc0138b8fabc209247f72d758e1cf3343756d543badbf24212bed8c15'

In [6]:
import numpy as np

In [21]:
arr = np.array([1, 2, 150], dtype=np.int32)

In [24]:
hex(arr[1])

'0x2'

In [8]:
from web3 import Web3
from solcx import compile_files
import merkletools

In [9]:
mt = merkletools.MerkleTools(hash_type="keccak_256")

mt.add_leaf(["hello", "world", "Kusrp" "bitcoin", "blockchain"], True)

mt.make_tree()

In [11]:
mt.get_proof(1)

[{'left': '1c8aff950685c2ed4bc3174f3472287b56d9517b9c948127319a09a7a36deac8'},
 {'right': '826299fd0ac1be3d3ef42f928ac65dbdaa24a58042aaab28f437c45e70707f6b'}]

In [12]:
contract_id, compiled_sol = compile_files(["verifyMerkleProof.sol"], output_values=['abi', 'bin']).popitem()

w3 = Web3(Web3.EthereumTesterProvider())

verifyMerkle = w3.eth.contract(abi=compiled_sol['abi'], bytecode=compiled_sol['bin'])

tx_hash = verifyMerkle.constructor().transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

merkleContract = w3.eth.contract(address=tx_receipt.contractAddress, abi=compiled_sol['abi'])

In [17]:
def proof_translate(mt_ins, x):
    root = '0x' + mt_ins.get_merkle_root()
    leaf = '0x' + mt_ins.get_leaf(x)
    res_proof = []
    res_bool = []
    for p in mt_ins.get_proof(x):
        if list(p.keys())[0] == 'left':
            res_proof.append('0x' + p['left'])
            res_bool.append(False)
        else:
            res_proof.append('0x' + p['right'])
            res_bool.append(True)
    return root, leaf, res_proof, res_bool


In [16]:
merkleContract.functions.verify(*proof_translate(mt, 1)).call()

False

In [63]:
encode_abi(['(int32,int32)'], [(23, 12)])

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x17\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0c'

In [46]:
def hash_op(op):
    combined = [0, 0, 0, 0]
    combined[0] = op['OpCode']
    combined[1:len(op['args'])+1] = op['args']
    
    return bytes(Web3.solidityKeccak(['int32', 'int32', 'int32', 'int32'], combined)).hex()

In [47]:
op1 = {"OpCode": 5, "args": []}

In [45]:
type(bytes(hash_op(op1)).hex())

str

In [48]:
hash_op(op1)

'a47f60972964e93c12e518b22ed4afe06aba169a6a3f473ffa14a4d10e4cd2c3'

In [52]:
def x2(x):
    return x*2
a = [1, 2, 3, 4]
b = list(map(x2, a))

In [54]:
a

[1, 2, 3, 4]

In [1]:
import ram
import numpy as np

source = {
    "size": 10,
    "code": [
        {
            "OpCode": 1,
            "args": [5, 2]
        },
        {
            "OpCode": 7,
            "args": []
        }
    ]
}


In [2]:
code = ram.RamCoder(10)
code.load(5, 0)
code.load(7, 1)
code.load(1, 5)
code.add(0, 2, 2)
code.sub(1, 5, 1)
code.jnz(1, 2)
code.end()

In [3]:
code.source

{'size': 10,
 'code': [{'OpCode': <OpCode.LOAD: 1>, 'args': [5, 0]},
  {'OpCode': <OpCode.LOAD: 1>, 'args': [7, 1]},
  {'OpCode': <OpCode.LOAD: 1>, 'args': [1, 5]},
  {'OpCode': <OpCode.ADD: 2>, 'args': [0, 2, 2]},
  {'OpCode': <OpCode.SUB: 3>, 'args': [1, 5, 1]},
  {'OpCode': <OpCode.JNZ: 6>, 'args': [1, 2]},
  {'OpCode': <OpCode.END: 7>, 'args': []}]}

In [4]:
ram1 = ram.Ram(code.source)

In [5]:
ram1()

In [11]:
import utils
import merkletools
leaves = list(map(utils.hash_rc, ram1.rc_list))
mt = merkletools.MerkleTools(hash_type="keccak_256")
mt.add_leaf(leaves)
mt.make_tree()
mt.get_merkle_root()

'b695a3cc8b9234a4e9e5ba89f99a301c36b56845736b4077cf063bfbef0efafd'

In [11]:
arr = np.zeros(source['size'], dtype=np.int32)

In [12]:
import utils
utils.store_mt(arr)

In [8]:
def fib_list(n):
    result = ['0', '1']
    for i in range(2, n+1):
        result.append(str(int(result[i-1]) + int(result[i-2])))
    return result


In [9]:
fib_list(32)

['0',
 '1',
 '1',
 '2',
 '3',
 '5',
 '8',
 '13',
 '21',
 '34',
 '55',
 '89',
 '144',
 '233',
 '377',
 '610',
 '987',
 '1597',
 '2584',
 '4181',
 '6765',
 '10946',
 '17711',
 '28657',
 '46368',
 '75025',
 '121393',
 '196418',
 '317811',
 '514229',
 '832040',
 '1346269',
 '2178309']